In [1]:
import pandas as pd
import numpy as np

In [2]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.sandbox.stats.multicomp import multipletests
import sys

sys.path.append('/Users/tsalo/Documents/tsalo/neurosynth/')
from neurosynth.analysis import stats


def p_to_z(p, sign):
    """
    From Neurosynth's meta.py
    """
    p = p/2  # convert to two-tailed
    # prevent underflow
    p[p < 1e-240] = 1e-240
    # Convert to z and assign tail
    z = np.abs(norm.ppf(p)) * sign
    # Set very large z's to max precision
    z[np.isinf(z)] = norm.ppf(1e-240)*-1
    return z


def decode(df, selected, prior=None, u=0.05):
    """
    Will require multiple comparisons correction.
    Can be used with BrainMap database as well.
    Taken from Neurosynth's meta-analysis method.
    
    Employs Benjamini-Hochberg FDR-correction
    """
    pmids = df.index.tolist()
    terms = df.columns.values
    unselected = list(set(pmids) - set(selected))
    
    sel_array = df.loc[selected].values
    unsel_array = df.loc[unselected].values
    
    n_selected = len(selected)
    n_unselected = len(unselected)
    print n_selected, n_unselected
    
    n_selected_active = np.sum(sel_array, axis=0)
    n_unselected_active = np.sum(unsel_array, axis=0)
    print n_selected_active, n_unselected_active
    
    n_selected_inactive = n_selected - n_selected_active
    n_unselected_inactive = n_unselected - n_unselected_active
    
    n_active = n_selected_active + n_unselected_active
    n_inactive = n_selected_inactive + n_unselected_inactive
    
    pF = n_selected / (n_selected + n_unselected)
    pA = (n_selected_active + n_unselected_active) / (n_selected + n_unselected)
    pAgF = n_selected_active * 1.0 / n_selected  # p(term presence given in cluster)
    pAgU = n_unselected_active * 1.0 / n_unselected
    pFgA = pAgF * pF / pA  # Conditional probability (in cluster given term presence)

    # Recompute conditions with empirically derived prior
    if prior is None:
        prior = pF
    
    pAgF_prior = prior * pAgF + (1 - prior) * pAgU
    pFgA_prior = pAgF * prior / pAgF_prior
    
    # One-way chi-square test for consistency of activation
    p_fi = stats.one_way(n_selected_active, n_selected)
    sign_fi = np.sign(n_selected_active - np.mean(n_selected_active)).ravel()
    z_fi = p_to_z(p_fi, sign_fi)

    # Two-way chi-square test for specificity of activation
    cells = np.array([[n_selected_active, n_unselected_active],
                      [n_selected_inactive, n_unselected_inactive]]).T
    p_ri = stats.two_way(cells)
    sign_ri = np.sign(pAgF - pAgU).ravel()
    z_ri = p_to_z(p_ri, sign_ri)
        
    # FDR
    sig_fi, corr_fi, _, _ = multipletests(p_fi, alpha=u, method='fdr_bh',
                                          returnsorted=False)
    sig_ri, corr_ri, _, _ = multipletests(p_ri, alpha=u, method='fdr_bh',
                                          returnsorted=False)

    out_p = np.array([corr_fi, z_fi, pAgF_prior, corr_ri, z_ri, pFgA_prior]).T

    out_df = pd.DataFrame(columns=['pForward', 'zForward', 'probForward', 'pReverse', 'zReverse', 'probReverse'],
                          data=out_p)
    out_df['Term'] = terms
    out_df = out_df[['Term', 'pForward', 'zForward', 'probForward', 'pReverse', 'zReverse', 'probReverse']]
    return out_df

In [7]:
f1 = '/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/pmid-kwd.csv'
f2 = '/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/exp-mag.csv'

df1 = pd.read_csv(f1)
new_vals = {}
for i in df1.index:
    id_ = df1.loc[i].values[0]
    vals = [v for v in df1.loc[i].values[1:] if not pd.isnull(v)]
    new_vals[id_] = vals
all_terms = sorted(list(set([i for sl in new_vals.values() for i in sl])))
df3 = pd.DataFrame(index=new_vals.keys(), columns=all_terms, data=np.zeros((len(new_vals.keys()), len(all_terms)), int))

for i, vs in new_vals.items():
    for v in vs:
        df3.loc[i][v] = 1
df3.to_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/terms.csv', index_label='id')

In [8]:
df2 = pd.read_csv(f2)
df2['id'] = df2['PMID'].astype(str)
df2 = df2[['id', 'Cluster']]

df2.to_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/clusters.csv', index=False)

In [9]:
df4 = df2.join(df3, on='id', how='right')
count_df = df4.groupby('Cluster').sum()
count_df.index = ['Cluster {0}'.format(i) for i in count_df.index]
total_row = count_df.sum(numeric_only=True)
total_row.name = 'total'
count_df = count_df.append(total_row, ignore_index=False).transpose()
count_df.index.name = 'term'
count_df.to_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/term_counts.csv', index=True, index_label='cluster')

In [11]:
import pandas as pd

df1 = pd.read_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/clusters.csv')
df2 = pd.read_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/terms.csv', index_col='id')


cluster_nums = np.arange(1,7)
print cluster_nums

#for c in sorted(df1['Cluster'].unique()):
for c in cluster_nums:
    print c
    print df1.loc[df1['Cluster']==c]['id']
    sel_ids = df1.loc[df1['Cluster']==c]['id'].values
    print sel_ids
    p_df = decode(df2, sel_ids)
    p_df.to_csv('/Users/Katie/Dropbox/Data/Naturalistic/NewSolution/MAG{0}_pvalues.csv'.format(c), index=False)

[1 2 3 4 5 6]
1
0          9831460
2         15006683
6         16628606
7         16687157
11        20378581
18        22759716
21        24972303
23        25762672
37      12419129-2
52      15165355-2
70      15890534-3
73      15893474-2
74      15893474-3
75      15935262-1
76      15935262-2
78      16338048-1
79      16338048-2
80      16338048-3
90     17412611-10
92      17412611-2
95      17412611-5
97      17412611-7
98      17412611-8
99      17412611-9
102     17504783-3
106     17631871-3
107     18412134-1
112     18412134-6
156     19376237-1
162     19531876-1
          ...     
242     21382559-8
243     21382559-9
257     21861684-1
258     21861684-2
259     21861684-3
260     21861684-4
262     21972849-2
266     22019857-3
272     22110619-2
289     22516367-2
300     23202431-2
321     23616340-4
322     23616340-5
324     23616340-7
325     23616340-8
330     24015241-1
331     24015241-2
334     24015241-5
336     24015241-7
353     24583253-3
354     2481464